In [84]:
import whisper
from transformers import pipeline
from gtts import gTTS
import os
import numpy as np
import pyaudio

In [85]:
stt_model = whisper.load_model("small")

c:\Users\user1\.conda\envs\env\lib\site-packages\whisper\__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [86]:
nlp = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

c:\Users\user1\.conda\envs\env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [87]:
def load_instructions(file_path):
    instructions = {}
    with open(file_path, 'r') as file:
        for line in file:
            if ':' in line:
                key, value = line.strip().split(":", 1)
                instructions[key.strip()] = value.strip()
    return instructions

In [88]:
def process_input(user_input, instructions):
    labels = list(instructions.keys())
    result = nlp(user_input, labels)
    best_label = result['labels'][0]
    return instructions.get(best_label, "I'm sorry, I didn't understand that.")

In [89]:
def text_to_speech(text, output_file):
    tts = gTTS(text=text, lang='en')
    tts.save(output_file)

In [99]:
import wave
def record_audio(filename, duration=10, fs=44100):
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=fs, input=True, frames_per_buffer=1024)
    frames = []

    print("Recording...")
    for _ in range(0, int(fs / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)
    print("Recording finished.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()

In [115]:
instructions = load_instructions('instructions.txt')

In [121]:
record_audio("recording.wav")

Recording...
Recording finished.


In [122]:
result = stt_model.transcribe("recording.wav")
user_input_text = result['text']

In [123]:
print(user_input_text)

 My appointment is next week. Can you confirm my appointment? Or please tell me when I should come to the hospital again.


In [124]:
response_text = process_input(user_input_text, instructions)

In [125]:
output_audio_file = "response_output.mp3"
text_to_speech(response_text, output_audio_file)

In [126]:
os.system(f"mpg321 {output_audio_file}")

1

ImportError: cannot import name 'L3RawSocket' from 'scapy.all' (c:\Users\user1\.conda\envs\env\lib\site-packages\scapy\all.py)

In [7]:
from scapy.all import ARP, Ether, srp

def scan_network(network):
    """
    Scan the network to find all devices along with their IP addresses.
    """
    # Create ARP request packet
    arp_request = ARP(pdst=network)
    
    # Create Ethernet frame to broadcast the ARP request
    broadcast = Ether(dst="ff:ff:ff:ff:ff:ff")
    arp_request_broadcast = broadcast / arp_request
    
    # Send the packet and receive responses
    answered_list = srp(arp_request_broadcast, timeout=2, verbose=False)[0]
    
    devices = []

    for sent, received in answered_list:
        device_info = {
            "ip": received.psrc,
            "mac": received.hwsrc
        }
        try:
            device_info["device_name"] = socket.gethostbyaddr(received.psrc)[0]
        except socket.herror:
            device_info["device_name"] = "Unknown"
        
        devices.append(device_info)

    return devices

# Define the network range to scan
network = "192.168.1.0/24"

# Perform the scan
devices = scan_network(network)

# Print the list of devices
print("Available devices in the network:")
for device in devices:
    print(f"IP: {device['ip']}, MAC: {device['mac']}, Device Name: {device['device_name']}")


RuntimeError: Sniffing and sending packets is not available at layer 2: winpcap is not installed. You may use conf.L3socket orconf.L3socket6 to access layer 3